In [2]:
from dis import code_info
import dash
from dash import dcc, dash_table
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bootstrap_components as dbc
import yfinance as yf
from datetime import date, timedelta, datetime

import finance_lib as fb

import pandas as pd 
import numpy as np 
import talib as tb
import yfinance as yf

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from plotly.subplots import make_subplots
import datetime
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta

#from xgboost import XGBRegressor

In [3]:
def df_converter(df, df_sp500, df_dollar): 
    # removed sp50 & dollar testing
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

In [4]:
df_btc = yf.download('BTC-USD', progress=False)
df_sol = yf.download('SOL-USD', progress=False)

In [5]:
df_sp500 = yf.download('^GSPC', 
                    progress=False)
df_dollar = yf.download('DX=F',  
                    progress=False)
df_btc = df_converter(df_btc,df_sp500,df_dollar)

In [6]:
df_sp500 = yf.download('^GSPC', 
                    progress=False)
df_dollar = yf.download('DX=F',  
                    progress=False)
df_sol = df_converter(df_sol,df_sp500,df_dollar)

In [7]:
df_summary = pd.DataFrame(columns=['Coin', 'Percentage','Value'])
df_transactions = pd.DataFrame(columns=['Coin', 'Date', 'Percentage', 'Value', 'Signal'])

In [8]:
def portofolio (df_transactions, signal, value, date, df_coin, coin, df_summary):

    value_at_day = df_coin[df_coin.index == date]['Close'][0]
    percentage = value/value_at_day
    list_to_add = [coin, date, percentage, value, signal]
    df_length = len(df_transactions)
    df_transactions.loc[df_length] = list_to_add

    for coins in df_transactions['Coin'].unique():
        if coins == coin:
            if ~(df_summary['Coin'].str.contains(coins).any()):
                    for idx,percentage in enumerate(df_transactions['Percentage']):
                        df_summary.drop(df_summary.index[df_summary['Coin'] == coins], inplace=True)
                        df_length = len(df_summary)
                        actual_value = percentage * df_coin.tail(1)['Close'][0]
                        df_summary.loc[df_length] = [coins, percentage,actual_value]
            else:
                    value_to_add = 0
                    for idx,percentage in enumerate(df_transactions[df_transactions['Coin'] == coins]['Percentage']):
                        value_to_add += percentage
                        actual_value = value_to_add * df_coin.tail(1)['Close'][0]
                        df_summary.drop(df_summary.index[df_summary['Coin'] == coins], inplace=True)
                        df_length = len(df_summary)
                        df_summary.loc[df_length] = [coins, value_to_add,actual_value]

    return df_transactions, df_summary

In [9]:
data = pd.to_datetime('2022-05-24', format='%Y-%m-%d')
data2 = pd.to_datetime('2022-05-23', format='%Y-%m-%d')

In [10]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_btc,'BTC-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0


In [11]:
df_summary

,Coin,Percentage,Value
0,BTC-USD,0.000337,9.949169


In [12]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_btc,'BTC-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0


In [13]:
df_summary

,Coin,Percentage,Value
0,BTC-USD,0.000674,19.898338


In [14]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_sol,'SOL-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0


In [15]:
df_summary

,Coin,Percentage,Value
0,BTC-USD,0.000674,19.898338
1,SOL-USD,0.201396,9.170642


In [16]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_sol,'SOL-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0


In [17]:
df_summary

,Coin,Percentage,Value
0,BTC-USD,0.000674,19.898338
1,SOL-USD,0.402793,18.341284


In [18]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_sol,'SOL-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0
4,SOL-USD,2022-05-24,0.201396,10,0


In [19]:
df_summary

,Coin,Percentage,Value
0,BTC-USD,0.000674,19.898338
1,SOL-USD,0.604189,27.511927


In [20]:
df_near = yf.download('NEAR-USD', progress=False)

In [21]:
df_transactions, df_summary = portofolio(df_transactions,0,100,data,df_near,'NEAR-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0
4,SOL-USD,2022-05-24,0.201396,10,0
5,NEAR-USD,2022-05-24,16.771247,100,0


In [22]:
df_link = yf.download('LINK-USD', progress=False)
df_transactions, df_summary = portofolio(df_transactions,0,800,data,df_link,'LINK-USD',df_summary)
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0
4,SOL-USD,2022-05-24,0.201396,10,0
5,NEAR-USD,2022-05-24,16.771247,100,0
6,LINK-USD,2022-05-24,110.588810,800,0


In [23]:
def atualization(df_summary):
    value_list=[]
    for idx,coins in enumerate(df_summary['Coin'].unique()):
        df_coin = yf.download(coins, progress=False)
        value_to_add = df_coin.tail(1)['Close'][0] * df_summary[df_summary['Coin'] == coins]['Percentage'][idx]
        value_list.append(round(value_to_add,2))
    
    df_summary.drop('Value', axis=1, inplace=True)
    df_summary['Value'] = value_list
    return df_summary
        

In [24]:
df_summary = atualization(df_summary)
df_summary

,Coin,Percentage,Value
0,BTC-USD,0.000674,19.90
1,SOL-USD,0.604189,27.51
2,NEAR-USD,16.771247,91.02
3,LINK-USD,110.588810,746.94


In [25]:
import pandas as pd
import plotly.graph_objs as go

data = [
    go.Pie(labels=df_summary['Coin'],
           values=df_summary['Value'],
           hole=.4)
  ]

fig = go.Figure(px.pie(labels=df_summary['Coin'],
           values=df_summary['Value'],
           names = df_summary['Coin'],
           hole=.4,color_discrete_sequence=px.colors.qualitative.T10))
fig.update_layout(width=500, height=500,paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)', font_color = 'white')
fig.show()



In [26]:
df_transactions['Value'].sum()

950

In [28]:
df_transactions

,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0
4,SOL-USD,2022-05-24,0.201396,10,0
5,NEAR-USD,2022-05-24,16.771247,100,0
6,LINK-USD,2022-05-24,110.588810,800,0
